In [1]:
%load_ext autoreload
%autoreload 2
import pynwb
import os
import numpy as np

# DataJoint and DataJoint schema
import datajoint as dj

dj.config["filepath_checksum_size_limit"] = 1 * 1024**2

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    ElectrodeGroup,
    Electrode,
    FirFilterParameters,
    IntervalList,
    Session,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
)

from spyglass.lfp.v1 import (
    LFPElectrodeGroup,
    LFPSelection,
    LFPV1,
)
from spyglass.lfp.lfp_merge import LFPOutput


from spyglass.lfp.analysis.v1.lfp_band import LFPBandSelection,LFPBandV1
# from spyglass.lfp_band.lfp_band_merge import LFPBandOutput
import os
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)
warnings.simplefilter("ignore", category=UserWarning)

[2023-12-21 09:13:42,318][INFO]: Connecting sambray@lmf-db.cin.ucsf.edu:3306
[2023-12-21 09:13:42,392][INFO]: Connected sambray@lmf-db.cin.ucsf.edu:3306


## Define Animal Sessions

In [2]:
nwb_file_names = [
    #Olive
    "Olive20220624_.nwb",
    "Olive20220627_.nwb",
    "Olive20220628_.nwb",
    "Olive20220629_.nwb",
    "Olive20220630_.nwb",
    "Olive20220701_.nwb",
    "Olive20220702_.nwb",
    "Olive20220705_.nwb",
    "Olive20220706_.nwb",
    #Yoshi
    "Yoshi20220509_.nwb",
    "Yoshi20220510_.nwb",
    "Yoshi20220511_.nwb",
    "Yoshi20220519_.nwb",
    "Yoshi20220525_.nwb",
    #Winnie
    "Winnie20220712_.nwb",
    "Winnie20220713_.nwb",
    "Winnie20220714_.nwb",
    "Winnie20220715_.nwb",
    "Winnie20220716_.nwb",
    #Wallie
    "Wallie20220911_.nwb",
    "Wallie20220912_.nwb",
    "Wallie20220913_.nwb",
    "Wallie20220916_.nwb",
    #Totoro
    "Totoro20220524_.nwb",
    "Totoro20220525_.nwb",
    "Totoro20220526_.nwb",
    "Totoro20220527_.nwb",
    "Totoro20220530_.nwb",
    "Totoro20220531_.nwb",
    "Totoro20220602_.nwb",
    "Totoro20220603_.nwb",
    "Totoro20220606_.nwb",
    "Totoro20220621_.nwb",
    #Odins
    "Odins20220107_.nwb",
    "Odins20220110_.nwb",
    "Odins20220111_.nwb",
    "Odins20220113_.nwb",
    "Odins20220118_.nwb",
    "Odins20220119_.nwb",
    "Odins20220123_.nwb",
    "Odins20220124_.nwb",
    # "Odins20220126_.nwb", ???
    #Banner
    "Banner20211221_.nwb",
    "Banner20211222_.nwb",
    "Banner20211227_.nwb",
    "Banner20211228_.nwb",
    "Banner20211230_.nwb",
    "Banner20211231_.nwb",
    "Banner20220103_.nwb",
    "Banner20220105_.nwb",
    "Banner20220123_.nwb",
    "Banner20220124_.nwb",
    "Banner20220126_.nwb",
]

nwb_file_names.extend([
    #Olive
    "Olive20220707_.nwb",
    "Olive20220708_.nwb",
    "Olive20220711_.nwb",
    #Yoshi
    "Yoshi20220513_.nwb",
    "Yoshi20220516_.nwb",
    "Yoshi20220517_.nwb",
    "Yoshi20220518_.nwb",
    #Winnie
    "Winnie20220717_.nwb",
    "Winnie20220718_.nwb",
    "Winnie20220719_.nwb",
    "Winnie20220720_.nwb",
    "Winnie20220722_.nwb",
    #Wallie
    "Wallie20220914_.nwb",
    "Wallie20220915_.nwb",
    "Wallie20220922_.nwb",
    #Totoro
    "Totoro20220607_.nwb",
    "Totoro20220610_.nwb",
    "Totoro20220613_.nwb",
    "Totoro20220614_.nwb",
    "Totoro20220615_.nwb",
    "Totoro20220616_.nwb",
    "Totoro20220617_.nwb",
    #Odins
    "Odins20220226_.nwb",
    #Banner
    "Banner20220215_.nwb",
    "Banner20220216_.nwb",
    "Banner20220223_.nwb",
    "Banner20220224_.nwb",
    "Banner20220225_.nwb",
])
    
# np.isnan(np.mean([]))
animal_names = ["Wallie","Winnie","Olive","Totoro","Yoshi","Dan","Bilbo","Frodo"]
from spyglass.common import Session
nwb_file_names = []
[nwb_file_names.extend((Session() & {"subject_id":animal}).fetch("nwb_file_name")) for animal in animal_names]

nwb_file_names = ['Winnie20220714_.nwb']

#write permission?
analysis_path = "/stelmo/nwb/analysis/"
for nwb_file in nwb_file_names:
    if not os.path.isdir(analysis_path+nwb_file.split('_')[0]):
        print(f'No analysis yet for: {nwb_file}')
    elif not os.access(analysis_path+nwb_file.split('_')[0],os.W_OK):
        print(f'No write permissions for: {nwb_file}nwb_file')


In [11]:
animal_names = ["Banner","Wallie","Winnie","Olive","Totoro","Yoshi","Dan","Bilbo","Frodo"]
nwb_file_names = []
interval_list_names = []

for animal in animal_names:
    nwb_file_names.extend((LFPV1 * Session() & {"subject_id":animal}).fetch("nwb_file_name"))
    interval_list_names.extend((LFPV1 * Session() & {"subject_id":animal}).fetch("target_interval_list_name"))

## Populate Ripple LFPBand

In [21]:
# nwb_file_names = ["Totoro20220602_.nwb",]
# interval_list_name = "pos 3 valid times"
filter_name = "Ripple 150-250 Hz"
# desired sampling rate
lfp_band_sampling_rate = 1000
missing_lfp = []



############################################
for nwb_file_name, interval_list_name in tqdm(zip(nwb_file_names, interval_list_names)):

    
    basic_key = {"nwb_file_name": nwb_file_name,'target_interval_list_name':interval_list_name}
    if LFPBandV1() & basic_key & {"filter_name":filter_name}:
        continue
    
    #check that lfp exists for this animal
    if len(LFPV1() & basic_key & {'lfp_sampling_rate':1000})==0:
        missing_lfp.append(basic_key)
        continue
    if len(LFPV1 & basic_key)>1:
        electrode_group_names = (LFPV1() & basic_key).fetch("lfp_electrode_group_name")
        if 'full_probe' in electrode_group_names:
            electrode_group_name = 'full_probe'
        else:
            electrode_group_name =[x for x in electrode_group_names if nwb_file_name.split('202')[0] in x][0]
        basic_key["lfp_electrode_group_name"] = electrode_group_name
        
    #get lfp's sampling rate
    lfp_sampling_rate = (LFPV1() & basic_key ).fetch1(
        "lfp_sampling_rate"
    )
    
    #get lfp filter sampling_rate
    lfp_filter_sampling_rate = (LFPV1() & basic_key).fetch1(
        "filter_sampling_rate"
    )
    
    #get lfp filter name
    lfp_filter_name = (LFPV1() & basic_key).fetch1(
        "filter_name"
    )
    
    # make lfp selection key
    lfp_s_key = basic_key.copy()
    lfp_s_key["target_interval_list_name"] = interval_list_name
    lfp_s_key["filter_name"] = lfp_filter_name
    lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
    
    # get the electrodes the lfp was run on
    lfp_electrode_group_name = (LFPV1 & lfp_s_key).fetch1('lfp_electrode_group_name')
    lfp_eg_key = {
            "nwb_file_name": nwb_file_name,
            "lfp_electrode_group_name": lfp_electrode_group_name,
            }
    electrode_list = list(np.asarray((LFPV1 & basic_key).fetch_nwb()[0]['lfp'].electrodes.to_dataframe().index))
    
    # remake lfp selection key
    lfp_s_key = basic_key.copy()
    lfp_s_key["target_interval_list_name"] = interval_list_name
    lfp_s_key["filter_name"] = lfp_filter_name
    lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
    
    # set the reference to -1 to indicate no reference for all channels
    ref_elect = [-1]
    
    # we also need the uuid for the LFP object
    lfp_merge_id = (LFPOutput.LFPV1 & lfp_s_key).fetch1('merge_id')
    
    
    # #make Band selection table entry
    LFPBandSelection().set_lfp_band_electrodes(
            nwb_file_name=nwb_file_name,
            lfp_merge_id=lfp_merge_id,
            electrode_list=electrode_list,
            filter_name=filter_name,
            interval_list_name=interval_list_name,
            reference_electrode_list=ref_elect,
            lfp_band_sampling_rate=lfp_band_sampling_rate,
        )
    
    lfp_band_key = (
        LFPBandSelection
        & {
            "lfp_merge_id": lfp_merge_id,
            "filter_name": filter_name,
            "lfp_band_sampling_rate": lfp_band_sampling_rate,
            "target_interval_list_name":interval_list_name,
        }
    ).fetch1("KEY")
    # if LFPBandV1() & lfp_band_key:
    #     continue #already populated
    # lfp_band_key['lfp_electrode_group_name']="full_probe"
LFPBandV1().populate(processes=4)#LFPBandSelection()&lfp_band_key)

0it [00:00, ?it/s][2023-12-21 09:25:27,796][WARNING]: Skipped checksum for file with hash: 6cd0fdb9-514d-d485-4dd4-bd1421c2a72f, and path: /stelmo/nwb/analysis/Banner20211221/Banner20211221_B8Y8EIB8TP.nwb


1it [00:00,  1.66it/s][2023-12-21 09:25:28,396][WARNING]: Skipped checksum for file with hash: 06436bdb-6d9c-22f6-029e-88c2f8c2412d, and path: /stelmo/nwb/analysis/Banner20211221/Banner20211221_FU6J57GS4G.nwb
2it [00:01,  1.64it/s][2023-12-21 09:25:29,015][WARNING]: Skipped checksum for file with hash: ff3b5c39-9487-8d16-a69c-43084265943b, and path: /stelmo/nwb/analysis/Banner20211222/Banner20211222_24ZSISUYXR.nwb
3it [00:01,  1.64it/s][2023-12-21 09:25:29,620][WARNING]: Skipped checksum for file with hash: 52940a6b-3512-2d5a-bd6d-715de04c937e, and path: /stelmo/nwb/analysis/Banner20211222/Banner20211222_NJ1AH2SG7X.nwb
4it [00:02,  1.64it/s][2023-12-21 09:25:30,232][WARNING]: Skipped checksum for file with hash: 18e8343e-3700-869b-51a4-ac5a8fae20c4, and path: /stelmo/nwb/analysis/Banner20211222/Banner20211222_PATV8XFL7F.nwb
5it [00:03,  1.66it/s][2023-12-21 09:25:30,823][WARNING]: Skipped checksum for file with hash: 9f22e680-257e-a70f-2030-352e0811be69, and path: /stelmo/nwb/analysis/

Writing new NWB file tonks20211023_RW4WULFA69.nwb


[2023-12-21 09:26:34,743][WARNING]: Skipped checksum for file with hash: 2f37ca57-0055-6b8c-ded4-0742803fb94e, and path: /stelmo/nwb/analysis/tonks20211101/tonks20211101_XOJY8AZAJI.nwb


Writing new NWB file tonks20211101_MEKNUYOFW9.nwb


[2023-12-21 09:26:36,387][WARNING]: Skipped checksum for file with hash: a4bd3988-a6d6-905c-c1f9-34c0b0c678a7, and path: /stelmo/nwb/analysis/Banner20220225/Banner20220225_TELGFY086W.nwb


Writing new NWB file tonks20211101_RBFH6JC8SF.nwb


[2023-12-21 09:26:38,724][WARNING]: Skipped checksum for file with hash: 0e9e9dc4-c16a-8085-bd72-7b44e397a4f5, and path: /stelmo/nwb/analysis/Banner20211227/Banner20211227_E0F1NKPG8U.nwb


Writing new NWB file tonks20211101_AOP38HKNOM.nwb


PermissionError: [Errno 13] Unable to create file (unable to open file: name = '/stelmo/nwb/analysis/tonks20211101/tonks20211101_AOP38HKNOM.nwb', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 242)

In [19]:
LFPV1 & {'nwb_file_name': 'Banner20220224_.nwb'}

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes,target_interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name name of the file,interval_list_name descriptive name of this interval list,lfp_object_id the NWB object ID for loading this object from the file,"lfp_sampling_rate the sampling rate, in HZ"
Banner20220224_.nwb,full_probe,pos 0 valid times,LFP 0-400 Hz,30000,Banner20220224_6CINFRHVDZ.nwb,lfp_full_probe_pos 0 valid times_valid times,1285ea58-f172-4256-95fb-3f9a78bb81d7,1000.0
Banner20220224_.nwb,full_probe,pos 1 valid times,LFP 0-400 Hz,30000,Banner20220224_QWRJ6I162L.nwb,lfp_full_probe_pos 1 valid times_valid times,fe7107d9-5acc-41bf-b6cd-483b72b97386,1000.0
Banner20220224_.nwb,full_probe,pos 2 valid times,LFP 0-400 Hz,30000,Banner20220224_LL4YF6DG4J.nwb,lfp_full_probe_pos 2 valid times_valid times,c62ceccd-b00f-4f6e-b162-228bc787b8bb,1000.0
Banner20220224_.nwb,full_probe,pos 3 valid times,LFP 0-400 Hz,30000,Banner20220224_MUWQXS2A9K.nwb,lfp_full_probe_pos 3 valid times_valid times,ade9a435-7296-48f6-88ff-0f15e2a4b765,1000.0
Banner20220224_.nwb,full_probe,pos 4 valid times,LFP 0-400 Hz,30000,Banner20220224_C4G2J48P31.nwb,lfp_full_probe_pos 4 valid times_valid times,db2e0a50-d1f5-4d63-8fa6-69a61168ea6c,1000.0
Banner20220224_.nwb,full_probe,pos 5 valid times,LFP 0-400 Hz,30000,Banner20220224_3ZY0DCTV6M.nwb,lfp_full_probe_pos 5 valid times_valid times,b5a6fa13-a96c-433a-a877-3963aad2a11e,1000.0
Banner20220224_.nwb,full_probe,pos 6 valid times,LFP 0-400 Hz,30000,Banner20220224_OLGSR49PFB.nwb,lfp_full_probe_pos 6 valid times_valid times,edbf32d6-9aa6-4dd1-a98e-e8921d202497,1000.0
Banner20220224_.nwb,full_probe,pos 7 valid times,LFP 0-400 Hz,30000,Banner20220224_91NGGQIP09.nwb,lfp_full_probe_pos 7 valid times_valid times,d162acd7-8c00-4ee5-8c67-9250f505a530,1000.0
Banner20220224_.nwb,test_group,pos 3 valid times,LFP 0-400 Hz,30000,Banner20220224_LP2O51GJ4J.nwb,lfp_test_group_pos 3 valid times_valid times,1e2be78c-cce4-41f4-989c-60c8db598342,1000.0
Banner20220224_.nwb,test_group,test interval,LFP 0-400 Hz,30000,Banner20220224_5MSHZ9CCOW.nwb,lfp_test_group_test interval_valid times,d44b3f90-724f-469c-8f8c-6a284f4535fa,1000.0


In [1]:
# LFPBandV1().populate(processes=4)]
import warnings 
from spyglass.lfp.analysis.v1.lfp_band import LFPBandV1
warnings.filterwarnings('ignore')
animal_names = ["Banner","Wallie","Winnie","Olive","Totoro","Yoshi","Dan","Bilbo","Frodo"]
for animal in animal_names:
    LFPBandV1().populate(f"nwb_file_name LIKE '{animal}%'",processes=8,display_progress=True)

[2023-12-21 09:43:42,659][INFO]: Connecting sambray@lmf-db.cin.ucsf.edu:3306
[2023-12-21 09:43:42,713][INFO]: Connected sambray@lmf-db.cin.ucsf.edu:3306
Processes:   0%|          | 0/80 [00:00<?, ?it/s]

Writing new NWB file Banner20211221_23QKBA128K.nwb
Writing new NWB file Banner20211221_P42YU5FP1O.nwb
Writing new NWB file Banner20211222_T0BU1WZMKM.nwb
Writing new NWB file Banner20211222_GB95P02BMR.nwb
Writing new NWB file Banner20211222_H49TEK6B8I.nwb
Writing new NWB file Banner20211222_9GGSZHOUZN.nwb
Writing new NWB file Banner20211222_FQFOYUGXUF.nwb
Writing new NWB file Banner20211227_92JAPRJETG.nwb


Processes:   1%|▏         | 1/80 [00:14<18:43, 14.22s/it]

Writing new NWB file Banner20211227_ZCV7WYSCTT.nwb
Writing new NWB file Banner20211227_8MM9QN52DB.nwb
Writing new NWB file Banner20211227_DDUVQV7YG0.nwb
Writing new NWB file Banner20211227_Y7MKEZMLKS.nwb
Writing new NWB file Banner20211228_QTQ75APIFX.nwb
Writing new NWB file Banner20211228_GC7SHY4XMY.nwb
Writing new NWB file Banner20211228_E5U4HLKHI7.nwb
Writing new NWB file Banner20211228_L66SXOZA8G.nwb


Processes:  11%|█▏        | 9/80 [00:25<02:56,  2.49s/it]

...

Processes:  89%|████████▉ | 71/80 [01:35<00:12,  1.36s/it]

Writing new NWB file Banner20220225_5S1YI3FJWW.nwb
Writing new NWB file Banner20220225_CCVNK6YJ3J.nwb


Processes:  92%|█████████▎| 74/80 [01:37<00:06,  1.03s/it]

Writing new NWB file Banner20220225_4BLCWOU2LN.nwb


Processes: 100%|██████████| 80/80 [01:43<00:00,  1.29s/it]


## Populate Ripple Tables

In [2]:
import spyglass.ripple.v1 as sgrip
import spyglass.ripple.v1 as sgr
from spyglass.position import PositionOutput

# params = sgrip.RippleParameters().fetch("ripple_param_dict")[1]
# params_name = 'default_ms'
# sgr.RippleParameters().insert1({"ripple_param_dict":params,"ripple_param_name":params_name},skip_duplicates=True)

In [12]:
key_list = []
for animal in animal_names[:2]:
    table = (LFPBandV1 ) & f"nwb_file_name LIKE '{animal}%'" & "filter_name='Ripple 150-250 Hz'"
    # table.fetch1_dataframe()
    lfp_band_key_list = table.fetch("KEY")
    for lfp_band_key in lfp_band_key_list:
        if sgrip.RippleTimesV1() & lfp_band_key:
            continue
        if lfp_band_key['target_interval_list_name'] =='test interval':
            continue
        # from spyglass.common import RippleLFPSelection
        sgr.RippleLFPSelection().set_lfp_electrodes(lfp_band_key)
        rip_sel_key = (sgrip.RippleLFPSelection & lfp_band_key).fetch1("KEY")
        pos_key = (PositionOutput.TrodesPosV1() & {'nwb_file_name':lfp_band_key['nwb_file_name'],
                                'interval_list_name':lfp_band_key['target_interval_list_name']}).fetch1('KEY')
        
        key = {
            "ripple_param_name": "default_ms",
            **rip_sel_key,
            "pos_merge_id": pos_key["merge_id"],
        }
        key_list.append(key)

In [13]:
import warnings
warnings.filterwarnings('ignore')
sgrip.RippleTimesV1().populate(key_list,display_progress=True,processes=4)
# for key in key_list:
#     sgrip.RippleTimesV1().populate(key,display_progress=True,processes=1)

Processes:   0%|          | 0/79 [00:00<?, ?it/s]

Computing ripple times for: {'lfp_merge_id': UUID('638ce6b5-bf11-545a-c5da-8d4acae587e5'), 'filter_name': 'Ripple 150-250 Hz', 'filter_sampling_rate': 1000, 'nwb_file_name': 'Banner20211228_.nwb', 'target_interval_list_name': 'pos 7 valid times', 'lfp_band_sampling_rate': 1000, 'group_name': 'CA1', 'ripple_param_name': 'default_ms', 'pos_merge_id': UUID('05737968-0ed0-e8af-2c06-debc4c689bf0')}Computing ripple times for: {'lfp_merge_id': UUID('b7dc5423-249e-bcb8-7931-7f54f5dfafe3'), 'filter_name': 'Ripple 150-250 Hz', 'filter_sampling_rate': 1000, 'nwb_file_name': 'Banner20220223_.nwb', 'target_interval_list_name': 'pos 2 valid times', 'lfp_band_sampling_rate': 1000, 'group_name': 'CA1', 'ripple_param_name': 'default_ms', 'pos_merge_id': UUID('0838cfa6-0c73-9fa7-2ddc-adc67d7d88b2')}Computing ripple times for: {'lfp_merge_id': UUID('13fb8fd7-7aa5-7258-a6d9-8aa48e5182f7'), 'filter_name': 'Ripple 150-250 Hz', 'filter_sampling_rate': 1000, 'nwb_file_name': 'Banner20220103_.nwb', 'target_int

Processes:   1%|▏         | 1/79 [00:12<16:00, 12.31s/it]

Computing ripple times for: {'lfp_merge_id': UUID('2a9092d8-35d9-6d94-d404-08fab51fa2b9'), 'filter_name': 'Ripple 150-250 Hz', 'filter_sampling_rate': 1000, 'nwb_file_name': 'Banner20211221_.nwb', 'target_interval_list_name': 'pos 1 valid times', 'lfp_band_sampling_rate': 1000, 'group_name': 'CA1', 'ripple_param_name': 'default_ms', 'pos_merge_id': UUID('0f37ed35-616e-2bba-ff50-5b1518679d45')}
Writing new NWB file Banner20211221_QA9VBPGHG1.nwb


Processes:   4%|▍         | 3/79 [00:13<04:28,  3.54s/it]

...

Processes:  97%|█████████▋| 77/79 [03:39<00:06,  3.12s/it]

Writing new NWB file Banner20211230_7967KDE42F.nwb


Processes:  99%|█████████▊| 78/79 [03:42<00:03,  3.01s/it]

Writing new NWB file Banner20211228_XNTXNJJJ0T.nwb


Processes: 100%|██████████| 79/79 [03:47<00:00,  2.88s/it]


In [64]:
%debug

> /home/sambray/mambaforge-pypy3/envs/spyglass/lib/python3.9/site-packages/pandas/core/indexing.py(1589)_validate_integer()
   1587         len_axis = len(self.obj._get_axis(axis))
   1588         if key >= len_axis or key < -len_axis:
-> 1589             raise IndexError("single positional indexer is out-of-bounds")
   1590 
   1591     # -------------------------------------------------------------------

*** NameError: name 'key' is not defined
*** NameError: name 'key' is not defined
*** NameError: name 'key' is not defined
*** NameError: name 'self' is not defined
> /home/sambray/mambaforge-pypy3/envs/spyglass/lib/python3.9/site-packages/pandas/core/indexing.py(1656)_getitem_axis()
   1654 
   1655             # validate the location
-> 1656             self._validate_integer(key, axis)
   1657 
   1658             return self.obj._ixs(key, axis=axis)

*** NameError: name 'key' is not defined
> /home/sambray/mambaforge-pypy3/envs/spyglass/lib/python3.9/site-packages/pandas/core/in

In [62]:
import logging
logging.getLogger('datajoint').setLevel(logging.ERROR)

In [51]:
(sgrip.RippleTimesV1() & key).fetch1_dataframe()

,start_time,end_time
id,,
0,1.657749e+09,1.657749e+09
1,1.657749e+09,1.657749e+09
2,1.657749e+09,1.657749e+09
3,1.657749e+09,1.657749e+09
4,1.657749e+09,1.657749e+09
...,...,...
588,1.657750e+09,1.657750e+09
589,1.657750e+09,1.657750e+09
590,1.657750e+09,1.657750e+09
